In [108]:
import pandas as pd
from chefboost import Chefboost as chef

In [109]:
df = pd.read_csv('Training.csv')

In [110]:
df.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None


In [111]:
df['Delivery_Year'].value_counts() 

2017    6318
2018    5830
2016    4371
2019     414
2020      14
Name: Delivery_Year, dtype: int64

Puede ser que tomando los datos de 2017 para luego predecir 2018 posibilidad

In [112]:
df[['Delivery_Year', 'Delivery_Quarter']][df['Delivery_Year'] == 2018]['Delivery_Quarter'].value_counts()

Q2    1582
Q4    1435
Q1    1431
Q3    1382
Name: Delivery_Quarter, dtype: int64

# Conversion a dolar

In [113]:
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df['Total_Amount']*0.0096
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df['Total_Amount']*1.17
df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df['Total_Amount']*0.70
df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df['Total_Amount']*1.29
df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [114]:
df['Total_Amount'].value_counts()

5.952000e+01    513
4.224000e+01    460
1.728000e+02    351
1.344000e+03    272
1.536000e+01    204
               ... 
1.075200e+04      1
6.869055e+05      1
3.163680e+04      1
1.373772e+06      1
1.749888e+06      1
Name: Total_Amount, Length: 9350, dtype: int64

In [115]:
df_short = df[['Region', 'Total_Amount', 'Delivery_Year', 'Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr','Stage' ]].rename(columns={'Stage': 'Decision'})
df_short = df_short[(df_short['Decision'] == 'Closed Lost') | (df_short['Decision'] == 'Closed Won')]
df_short.head()

,Region,Total_Amount,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,EMEA,6169176.0,2016,Q2,1,1,Closed Lost
1,EMEA,56429.1,2016,Q1,0,0,Closed Won
2,Americas,83865.6,2016,Q1,0,0,Closed Won
3,Americas,7421881.5,2018,Q1,0,1,Closed Lost
4,Americas,13357192.5,2018,Q1,0,1,Closed Lost


In [116]:
training = df_short.drop(df_short[(df_short['Delivery_Year']== 2018) & (df_short['Delivery_Quarter'] == 'Q4')].index).drop(df_short[(df_short['Delivery_Year'] > 2018)].index)
training['Delivery_Year'].value_counts()

2017    6317
2018    4395
2016    4371
Name: Delivery_Year, dtype: int64

In [117]:
test = df_short[(df_short['Delivery_Year']== 2018) & (df_short['Delivery_Quarter'] == 'Q4')]
test.head()

,Region,Total_Amount,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
956,EMEA,2204771.40,2018,Q4,1,1,Closed Lost
1000,APAC,9567090.00,2018,Q4,0,1,Closed Lost
1001,APAC,9567360.00,2018,Q4,0,1,Closed Lost
1436,APAC,5099864.00,2018,Q4,0,1,Closed Lost
2559,APAC,1531920.25,2018,Q4,0,1,Closed Lost


In [118]:
config = {'algorithm' : 'C4.5'}

In [119]:
model = chef.fit(training.copy(), config)

C4.5  tree is going to be built...
-------------------------
finished in  5.3954432010650635  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  76.58290790956707 % on  15083  instances
Labels:  ['Closed Lost' 'Closed Won']
Confusion matrix:  [[3735, 817], [2715, 7816]]
Precision:  82.0518 %, Recall:  57.907 %, F1:  67.8967 %


In [120]:
i = 0;
j = 0;
for index, instance in test.iterrows():
    prediction = chef.predict(model, instance)
    actual = instance['Decision']
    i = i + 1
    
    if actual == prediction:
        classified = True
        j = j + 1
    else:
        classified = False

prediccion = j/i
print(prediccion)

0.6390243902439025


# Gradient Boosting

In [121]:
config = {'enableGBM': True, 'epochs': 7, 'learning_rate': 1, 'max_depth': 5}

In [ ]:
model = chef.fit(training.copy(), config)

Boosting:   0%|          | 0/7 [00:00<?, ?it/s]

Gradient Boosting Machines...
Regression  tree is going to be built...
gradient boosting for classification


In [ ]:
i = 0;
j = 0;
for index, instance in test.iterrows():
    prediction = chef.predict(model, instance)
    actual = instance['Decision']
    i = i + 1
    
    if actual == prediction:
        classified = True
        j = j + 1
    else:
        classified = False

prediccion = j/i
print(prediccion)

# Random Forest

In [ ]:
config = {'enableRandomForest': True, 'num_of_trees': 5}

In [ ]:
model = chef.fit(training.copy(), config)

In [ ]:
i = 0;
j = 0;
for index, instance in test.iterrows():
    prediction = chef.predict(model, instance)
    actual = instance['Decision']
    i = i + 1
    
    if actual == prediction:
        classified = True
        j = j + 1
    else:
        classified = False

prediccion = j/i
print(prediccion)